## Data Handling

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
np.random.seed(4444)

In [2]:
df = pd.read_csv('../../../../../jaeyeun/01_nh_poc/16_re_label/')

df.head()

FileNotFoundError: [Errno 2] File ../../../../../../jaeyeun/01_nh_poc/16_re_label/split_70_15_15_re_2.csv does not exist: '../../../../../../jaeyeun/01_nh_poc/16_re_label/split_70_15_15_re_2.csv'

* only for kdi dataset(exp1) & label_3

In [ ]:
df = df[df['dataset_n'] == 0]

In [3]:
X_train_df = df[df['train_val_test'] == 0]
X_val_df = df[df['train_val_test'] == 1]
X_test_df = df[df['train_val_test'] == 2]
y_train = df[df['train_val_test'] == 0]['label_3']
y_val = df[df['train_val_test'] == 1]['label_3']
y_test = df[df['train_val_test'] == 2]['label_3']

NameError: name 'df' is not defined

In [4]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

NameError: name 'X_train_df' is not defined

In [6]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
    ngram_range = (1, 3),
    max_features=n_features)

* data is 'prep_v9_text'

In [7]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['prep_v9_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [8]:
X_train = X_train_tfidf.transform(X_train_df['prep_v9_text'])
X_val = X_train_tfidf.transform(X_val_df['prep_v9_text'])
X_test = X_train_tfidf.transform(X_test_df['prep_v9_text'])

In [9]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(7555, 100000)
(1621, 100000)
(1624, 100000)


## Classify

* LightGBM RandomSearch

In [10]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgbm

In [ ]:
lgbm_clf = lgbm.LGBMClassifier()

lgbm_param_grid = {
#     'gamma': [.05, .1, .3, .5, .7, .9, 1],

    # Core params
    'boosting': ['gbdt', 'goss'],
    'learning_rate': [.01, .015, .025, .05, .1, .125, .15],
    'num_leaves' : [5, 15, 30, 50, 100, 150, 200, 400], #it must be smaller than 2^(max_depth)
    'device_type' : ['gpu'],
    
    # Learning Control params
    'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
    'min_data_in_leaf': [10, 15, 20, 30, 50, 100],
    'min_sum_hessian_in_leaf': [.0001, .0005,.001, .002, .005, .01],
    'bagging_fraction': np.linspace(0.6, 1, 5), # aliases subsample
    'bagging_freq': range(0,10),
    'feature_fraction': np.linspace(0.6, 1, 5),

    # Dataset params
    'max_bin': [16, 32, 64, 128, 256]
}

fit_params={
    'early_stopping_rounds' : 30,
    'eval_metric' : "multi_error",
    'eval_set' : [(X_train, y_train), (X_val, y_val)]
}


# Create a random search object
lgbm_random = RandomizedSearchCV(estimator = lgbm_clf,
                                param_distributions = lgbm_param_grid,
                                n_iter = 30, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=24,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
lgbm_random.fit(X_train, y_train, **fit_params)

In [1]:
pred_train = lgbm_random.best_estimator_.predict(X_train)
pred_val = lgbm_random.best_estimator_.predict(X_val)
pred_test = lgbm_random.best_estimator_.predict(X_test)

NameError: name 'lgbm_random' is not defined

In [12]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.9917334022216482
0.8324291742013261
0.8576555023923444


In [14]:
cv_result_df = pd.DataFrame(lgbm_random.cv_results_)

df_list = []
for i in range(3):
    df_list.append(pd.DataFrame([cv_result_df.loc[i, "params"]]))
    
param_table = pd.concat(df_list)

param_table['mean_test_score'] = cv_result_df['mean_test_score'].values

param_table.sort_values(by='mean_test_score', axis=0)

,num_leaves,max_depth,mean_test_score
0,10,3,0.839315
0,10,5,0.847968
0,15,5,0.849777
